# 1. 감성분석의 이해

감성 분석 방법론은 다양하게 분류될 수 있는데, 크게는 어휘 기반의 분석과 기계학습 기반의 분석으로 나뉘어진다.

## 1.1 어휘 기반의 감성 분석

어휘 기반의 감성 분석은 감성이 표현될 수 있는 명사, 형용사, 동사를 대상으로 모든 단어에 대해 긍정 혹은 부정의 감성을 붙여서 감성 사전을 구축한 후에, 이를 기반으로 텍스트에 대한 감성 분석을 수행하는 방식이다.

## 1.2 머신러닝 기반의 감성 분석

학습 데이터셋만 있다면 문서분류에서 사용한 다양한 알고리즘으로 학습을 통해 감성을 예측할 수 있다.

# 2. 감성 사전을 이용한 영화 리뷰 감성 분석

## 2.1 NLTK 영화 리뷰 데이터 준비

In [2]:
import nltk
nltk.download('movie_reviews')

from nltk.corpus import movie_reviews

print('#review count:', len(movie_reviews.fileids())) #영화 리뷰 문서의 id를 반환
print('#samples of file ids:', movie_reviews.fileids()[:10]) #id를 10개까지만 출력
print('#categories of reviews:', movie_reviews.categories()) # label, 즉 긍정인지 부정인지에 대한 분류
print('#num of "neg" reviews:', len(movie_reviews.fileids(categories='neg'))) #label이 부정인 문서들의 id를 반환
print('#num of "pos" reviews:', len(movie_reviews.fileids(categories='pos'))) #label이 긍정인 문서들의 id를 반환

fileid = movie_reviews.fileids()[0] #첫번째 문서의 id를 반환
print('#id of the first review:', fileid)
print('#part of the first review:', movie_reviews.raw(fileid)[:500]) #첫번째 문서의 내용을 500자까지만 출력
print('#sentiment of the first review:', movie_reviews.categories(fileid)) #첫번째 문서의 감성

fileids = movie_reviews.fileids() #movie review data에서 file id를 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids] #file id를 이용해 raw text file을 가져옴
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids]

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /data/ydkim/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


#review count: 2000
#samples of file ids: ['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt', 'neg/cv005_29357.txt', 'neg/cv006_17022.txt', 'neg/cv007_4992.txt', 'neg/cv008_29326.txt', 'neg/cv009_29417.txt']
#categories of reviews: ['neg', 'pos']
#num of "neg" reviews: 1000
#num of "pos" reviews: 1000
#id of the first review: neg/cv000_29416.txt
#part of the first review: plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
what's the deal ? 
watch the movie and " sorta " find out . . . 
critique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . 
which is what makes this review an even harder one to write , since i generally applaud films which attempt
#sentiment of the first review: ['neg']


## 2.2 TextBlob을 이용한 감성 분석

In [3]:
pip install -U textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 KB 23.4 MB/s eta 0:00:00
You should consider upgrading via the '/data/ydkim/.pyenv/versions/3.8.12/envs/py3.8.12/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /data/ydkim/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /data/ydkim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /data/ydkim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /data/ydkim/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /data/ydkim/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /data/ydkim/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


첫째 리뷰에 대한 결과값

In [5]:
from textblob import TextBlob

result = TextBlob(reviews[0])
print(result.sentiment)

Sentiment(polarity=0.06479782948532947, subjectivity=0.5188408350908352)


polarity는 -1과 1사이의 실수로 극성을 나타내는 반면, 우리가 가진 데이터에서는 pos와 neg로 극성을 표현하므로, 데이터에 맞게 반환하는 함수를 아래와 같이 작성한다.

In [6]:
def sentiment_TextBlob(docs):
    results = []
    
    for doc in docs:
        testimonial = TextBlob(doc)
        if testimonial.sentiment.polarity > 0:
            results.append('pos')
        else:
            results.append('neg')
            
    return results

사이킷런에서 accuracy_score를 불러와서 아래와 같이 정확도를 확인한다.

In [7]:
from sklearn.metrics import accuracy_score

print('#TextBlob을 이용한 리뷰 감성분석의 정확도: ', accuracy_score(categories, sentiment_TextBlob(reviews)))

#TextBlob을 이용한 리뷰 감성분석의 정확도:  0.6


생각보다 정확도가 좋지 않은 것을 볼 수 있다. 감성사전을 기반으로 하는 감성 분석의 가장 큰 문제는 이와 같이 정확도가 그리 높지 않다는 점이다.

## 2.3 AFINN을 이용한 감성 분석

AFINN 어휘 목록은 -5에서 5사이의 극성을 부여한 영어 단어들의 리스트다.

In [8]:
pip install afinn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 KB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53448 sha256=182f3452f08743175bdac1ad835bb7448c79414f6c20fdd871c9af36426f9926
  Stored in directory: /data/ydkim/.cache/pip/wheels/f6/6f/c3/b305c5107a17618f2938a067d5ffcbb556909d82398762089e
Successfully built afinn
You should consider upgrading via the '/data/ydkim/.pyenv/versions/3.8.12/envs/py3.8.12/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
from afinn import Afinn

def sentiment_Afinn(docs):
    afn = Afinn(emoticons=True)
    results = []
    
    for doc in docs:
        if afn.score(doc) > 0:
            results.append('pos')
        else:
            results.append('neg')
            
    return results


print('#Afinn을 이용한 리뷰 감성분석의 정확도: ', accuracy_score(categories, sentiment_Afinn(reviews)))

#Afinn을 이용한 리뷰 감성분석의 정확도:  0.664


## 2.4 VADER를 이용한 감성 분석

VADER는 규칙기반의 감성 분석 알고리즘을 사용하는 것으로 알려져 있다.

In [11]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /data/ydkim/nltk_data...


True

vader의 SentimentIntensityAnalyzer는 4개의 극성값을 제공한다. pos, neg, neu, compound. compound는 나머지 세 값을 적절히 조합해 -1과 1 사이의 극성값을 제공한다.

In [12]:
from afinn import Afinn
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def sentiment_vader(docs):
    analyser = SentimentIntensityAnalyzer()
    results = []
    
    for doc in docs:
        score = analyser.polarity_scores(doc)
        if score['compound'] > 0:
            results.append('pos')
        else:
            results.append('neg')
            
    return results
print('#Vader를 이용한 리뷰감성분석의 정확도: ', accuracy_score(categories, sentiment_vader(reviews)))

#Vader를 이용한 리뷰감성분석의 정확도:  0.635


# 3. 학습을 통한 머신러닝 기반의 감성분석

## 3.1 NLTK 영화 리뷰에 대한 머신러닝 기반 감성 분석

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews, categories, test_size=0.2, random_state=7)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB #sklearn이 제공하는 MultinomialNB 를 사용

tfidf = TfidfVectorizer().fit(X_train) 

X_train_tfidf = tfidf.transform(X_train) # train set을 변환
print('#Train set dimension:', X_train_tfidf.shape) # 실제로 몇개의 특성이 사용되었는지 확인
X_test_tfidf = tfidf.transform(X_test) # test set을 변환
print('#Test set dimension:', X_test_tfidf.shape)

NB_clf = MultinomialNB(alpha=0.01) # 분류기 선언
NB_clf.fit(X_train_tfidf, y_train) #train set을 이용하여 분류기(classifier)를 학습
print('#Train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf, y_train))) #train set에 대한 예측정확도를 확인
print('#Test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf, y_test))) #test set에 대한 예측정확도를 확인

#Train set dimension: (1600, 36189)
#Test set dimension: (400, 36189)
#Train set score: 0.998
#Test set score: 0.797
